In [1]:
#Dependents
import pandas as pd
import os
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np

#API Key
from api_keys import g_key

In [2]:
#Read in DataFrame from part 1
citydata_df = pd.read_csv(r'C:\Users\Cody\UCBE\python-api-challenge\clean_city_output.csv')
citydata_df.head()

,City,Lat,Long,Max Tempurature,Humidity,Cloudiness,Wind Speed,Country,Date
0,carnarvon,-24.8667,113.6333,77.00,94,75,12.66,AU,1611268769
1,mataura,-46.1927,168.8643,57.99,81,69,13.00,NZ,1611268770
2,bethel,41.3712,-73.4140,32.00,64,20,3.98,US,1611268770
3,yerbogachen,61.2767,108.0108,-44.81,78,48,3.27,RU,1611268771
4,seoul,37.5683,126.9778,39.20,100,75,4.61,KR,1611268615


In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Humidity heatmap
location = citydata_df[["Lat", "Long"]]
humidity = citydata_df["Humidity"]
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(location, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heatmap_layer)

print(fig)

Figure(layout=FigureLayout(height='420px'))


In [5]:
cleancity_df = citydata_df.loc[(citydata_df["Max Tempurature"] < 80) & (citydata_df["Max Tempurature"] > 65) \
                              & (citydata_df["Wind Speed"] < 8) \
                              & (citydata_df["Cloudiness"] == 0)].dropna()

cleancity_df.head()

,City,Lat,Long,Max Tempurature,Humidity,Cloudiness,Wind Speed,Country,Date
189,diego de almagro,-26.3667,-70.0500,72.27,27,0,6.91,CL,1611268839
278,pilane,-24.4500,26.1333,77.00,61,0,6.91,BW,1611268865
339,qeshm,26.9581,56.2719,66.20,67,0,4.61,IR,1611268885
342,salalah,17.0151,54.0924,68.00,60,0,3.44,OM,1611268886
420,bhiwandi,19.3000,73.0667,73.40,68,0,2.55,IN,1611268912


In [6]:
# New DF to store  hotel names and location
hotel_df = cleancity_df[["City", "Country", "Lat", "Long"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(5)

,City,Country,Lat,Long,Hotel Name
189,diego de almagro,CL,-26.3667,-70.0500,
278,pilane,BW,-24.4500,26.1333,
339,qeshm,IR,26.9581,56.2719,
342,salalah,OM,17.0151,54.0924,
420,bhiwandi,IN,19.3000,73.0667,


In [7]:
#Set hotel search paramaters
paramaters = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iteration
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    long = row["Long"]
    
    paramaters["location"] = f"{lat},{long}"
    
    #Create a base url using "Hotel" search and lat/long
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Request api and print results
    hotel_address = requests.get(base_url, params=paramaters)
    
    #Convert to json format
    hotel_address = hotel_address.json()
    
    #Get first hotel and store in DF
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result...skipping.")
hotel_df.head(30)

Missing result...skipping.


,City,Country,Lat,Long,Hotel Name
189,diego de almagro,CL,-26.3667,-70.0500,hotel luna del desierto
278,pilane,BW,-24.4500,26.1333,
339,qeshm,IR,26.9581,56.2719,Qeshm Kavan Hotel
342,salalah,OM,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
420,bhiwandi,IN,19.3000,73.0667,OYO 7136 Solitaire Residency
430,geraldton,AU,-28.7667,114.6000,Broadwater Mariner Resort
445,cowra,AU,-33.8333,148.6833,Cowra Motor Inn
452,nelson bay,AU,-32.7167,152.1500,Mantra Nelson Bay
461,maryborough,AU,-25.5333,152.7000,Blue Shades Motel
524,orodara,BF,10.9492,-4.9342,Bkf Orodara


In [8]:
#Using template to add hotel marks to heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [10]:
#Adding a marker layer to heatmap
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


print(fig)

Figure(layout=FigureLayout(height='420px'))
